In [ ]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import queue
import threading
import re
import time
import os

def fetch_data_with_retry(resource_id, offset, limit=1000, retries=3):
    base_url = f"https://opendata.nhsbsa.net/api/3/action/datastore_search?offset={offset}&q=%7B%22ICB_CODE%22%3A%22QMM%3A%2A%22%7D&resource_id={resource_id}&limit={limit}"
    while retries > 0:
        try:
            response = requests.get(base_url)
            if response.status_code == 200:
                data = response.json()
                records = data['result']['records']
                return records
            elif response.status_code == 502:
                print(f"Server error 502 with offset {offset}. Retrying after 5 seconds...")
                time.sleep(5)
                retries -= 1
            elif response.status_code == 500:
                print(f"Server error 500 with offset {offset}.")
                return []
            else:
                print(f"Error {response.status_code} with offset {offset}.")
                return []
        except requests.RequestException as e:
            print(f"Request exception with offset {offset}: {e}")
            return []
    print(f"Failed to fetch data with offset {offset} after retries.")
    return []

def fetch_records_for_range(resource_id, offset_queue, all_records, lock, limit, stop_event):
    while not offset_queue.empty() and not stop_event.is_set():
        offset = offset_queue.get()
        records = fetch_data_with_retry(resource_id, offset, limit)
        if not records:
            stop_event.set()  # Stop signal for all workers if no records are returned
            print(f"No records found for offset {offset}. Signaling to stop further fetching.")
        else:
            with lock:
                all_records.extend(records)
        offset_queue.task_done()

def get_all_data_concurrently(resource_id, limit=1000):
    all_records = []
    offsets_to_fetch = queue.Queue()
    stop_event = threading.Event()  # Stop signal for all workers
    
    # Prepopulate the offsets from 0 to 1,000,000 in steps of 1,000
    for offset in range(0, 1000000, limit):
        offsets_to_fetch.put(offset)
    
    lock = threading.Lock()

    # Use ThreadPoolExecutor to fetch offsets concurrently
    with ThreadPoolExecutor(max_workers=16) as executor:
        futures = []
        for _ in range(16):
            future = executor.submit(fetch_records_for_range, resource_id, offsets_to_fetch, all_records, lock, limit, stop_event)
            futures.append(future)
        for future in as_completed(futures):
            future.result()

    return all_records

def main():
    # Set YYYYMM data set to pull
    year_month = 202405
    
    # Ensure the output directory exists
    output_directory = "Dispensing Data"
    os.makedirs(output_directory, exist_ok=True)
    
    resource_id = f"EPD_{year_month}"
    print(f"Fetching data for {resource_id}")
    
    all_data = get_all_data_concurrently(resource_id)
    df = pd.DataFrame(all_data)

    # Define the column order
    column_order = [
        'YEAR_MONTH', 'ICB_CODE', 'ICB_NAME', 'PCO_CODE', 'PCO_NAME', 
        'REGIONAL_OFFICE_CODE', 'REGIONAL_OFFICE_NAME', 'PRACTICE_CODE', 
        'PRACTICE_NAME', 'ADDRESS_1', 'ADDRESS_2', 'ADDRESS_3', 'ADDRESS_4', 
        'POSTCODE', 'BNF_CHAPTER_PLUS_CODE', 'BNF_CHEMICAL_SUBSTANCE', 
        'CHEMICAL_SUBSTANCE_BNF_DESCR', 'BNF_CODE', 'BNF_DESCRIPTION', 
        'QUANTITY', 'ITEMS', 'TOTAL_QUANTITY', 'ADQUSAGE', 'NIC', 
        'ACTUAL_COST', 'UNIDENTIFIED'
    ]

    # Reorder the columns in the DataFrame
    df = df[column_order]

    # Create the output file name based on the resource_id
    yyyymm = str(year_month)
    file_name = f"{yyyymm} - N&WICS Prescribing Data.xlsx"
    file_path = os.path.join(output_directory, file_name)

    # Save the DataFrame to an Excel file
    df.to_excel(file_path, index=False)
    print(f"Data has been fetched and saved to {file_path}")


if __name__ == "__main__":
    main()


Fetching data for EPD_202205


Finished processing offset 13000


Finished processing offset 12000
Finished processing offset 5000
Finished processing offset 11000
Finished processing offset 0


Finished processing offset 10000
Finished processing offset 6000
Finished processing offset 4000
Finished processing offset 3000
Finished processing offset 9000
Finished processing offset 1000


Finished processing offset 14000
Finished processing offset 7000
Finished processing offset 2000
Finished processing offset 15000


...

Finished processing offset 18000
Finished processing offset 16000
Finished processing offset 21000
Finished processing offset 25000
Finished processing offset 17000


Finished processing offset 29000
Finished processing offset 27000
Finished processing offset 31000


Finished processing offset 20000
Finished processing offset 23000
Finished processing offset 26000


Finished processing offset 24000


Finished processing offset 32000
Finished processing offset 28000
Finished processing offset 22000
